In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import uniform, randint

In [2]:
# Loading data
X = pd.read_csv("~/uk-pol-speech-classifier/notebooks/features_1000sample_400min_600cutoff.csv")
y = pd.read_csv("~/uk-pol-speech-classifier/notebooks/target_1000sample_400min_600cutoff.csv")

In [3]:
X

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,1,0.034373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.091811,0.0,0.0,0.0,0.0,0.0
4,4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,6995,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6996,6996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6997,6997,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6998,6998,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [4]:
X = X.drop(columns="Unnamed: 0")

In [5]:
# Convert one-hot encoded target array back to class labels
y = y.drop(columns="Unnamed: 0")
y_not_encoded = y.columns.get_indexer(y.idxmax(1))+1
np.unique(y), y

(array([0., 1.]),
       party_Con  party_DUP  party_Lab  party_LibDem  party_PlaidCymru  \
 0           1.0        0.0        0.0           0.0               0.0   
 1           1.0        0.0        0.0           0.0               0.0   
 2           1.0        0.0        0.0           0.0               0.0   
 3           1.0        0.0        0.0           0.0               0.0   
 4           1.0        0.0        0.0           0.0               0.0   
 ...         ...        ...        ...           ...               ...   
 6995        0.0        0.0        0.0           0.0               0.0   
 6996        0.0        0.0        0.0           0.0               0.0   
 6997        0.0        0.0        0.0           0.0               0.0   
 6998        0.0        0.0        0.0           0.0               0.0   
 6999        0.0        0.0        0.0           0.0               0.0   
 
       party_SNP  party_UUP  
 0           0.0        0.0  
 1           0.0        0.0  
 2

In [6]:
y_not_encoded

array([1, 1, 1, ..., 7, 7, 7])

In [7]:
# Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X, y_not_encoded, test_size=0.2, random_state=42)

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'C': uniform(0.1, 10),  # Penalty parameter C (regularization parameter)
    'gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'
    'degree': randint(1, 10)  # Degree of the polynomial kernel function ('poly')
}

# Create an SVM classifier
svm_classifier = SVC(kernel="poly")

# Perform random search cross-validation
random_search = RandomizedSearchCV(svm_classifier, param_distributions=param_grid, n_iter=20, scoring='accuracy', cv=5, n_jobs=-1)
random_search.fit(X_train, y_train)

# Best hyperparameters found
print("Best Hyperparameters:", random_search.best_params_)

# Best cross-validation score
print("Best Cross-validation Score:", random_search.best_score_)

# Evaluate the best model on test data
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Data:", accuracy)

Best Hyperparameters: {'C': 9.091651261779795, 'degree': 1, 'gamma': 'scale'}
Best Cross-validation Score: 0.6283928571428572
Accuracy on Test Data: 0.65


In [10]:
model = SVC(kernel="poly", degree=1, gamma="scale", C=9.1)
model.fit(X_train, y_train)

SVC(C=9.1, degree=1, kernel='poly')

In [11]:
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.65
